In [1]:
!pip install openpyxl
!pip install pandas-gbq -U
!pip install tableone


In [2]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from tabulate import tabulate
from google.oauth2 import service_account
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms

from tableone import TableOne
import pandas as pd
import pandas_gbq
import pydata_google_auth
import matplotlib.pyplot as plt
import numpy as np
import math
import sklearn.metrics as metrics
from sklearn.calibration import calibration_curve

import warnings 
warnings.filterwarnings(action= 'ignore')

In [75]:
credentials = pydata_google_auth.get_user_credentials(['https://www.googleapis.com/auth/cloud-platform'],)
project = 'capacheiv'
pandas_gbq.context.credentials = credentials

In [204]:
query = ""
for fn in ['icustays', 'demographics', 'labs', 'cancer_diagnosis', 'pastcancer', 'apache_pr','apache_v', 'apache_data',
            'vital', 'admissionlabs', 'creatinine_labs', 'patient','hospital']:
    with open('sql/'+fn+'.sql', 'r') as file:
        query += file.read()
query += "select * from hospital"
project = 'capacheiv'

gbq_data = pd.read_gbq(query, dialect='standard', project_id=project)

In [207]:
data = gbq_data
df = data[((data['cancer_hemato'] == 1)|(data['cancer_non_hemato'] == 1))] 

In [208]:
columns = ['hospitaldischargeyear','age_num', 'gender', 'teachingstatus', 'numbedscategory', 'ethnicity',
           'has_sepsis', 'cancer_hemato', 'cancer_non_hemato', 'metastasis',
           'renal_GU', 'chest_breast','lung_pulmonary', 'mss_skin', 'head_neck', 'GI', 'CNS', 'endocrine',
           'vent', 'vent_upon_admission', 'vent_during_hosp', 'min_wbc', 'max_wbc', 'min_creatinine_day1', 'max_creatinine_day1',
           'min_creatinine_day_3','max_creatinine_day_3', 'min_creatinine_day_5', 'max_creatinine_day_5', 
           'dialysis','vasopressor', 'asactualhospitallos', 'actualiculos', 'mortality', 'actualicumortality','apacheScore']



categorical = ['hospitaldischargeyear', 'gender', 'teachingstatus', 'numbedscategory', 'ethnicity', 
               'has_sepsis', 'cancer_non_hemato', 'cancer_hemato', 'metastasis',
               'renal_GU', 'chest_breast','lung_pulmonary', 'mss_skin', 'head_neck', 'GI', 'CNS', 'endocrine',
               'vent_upon_admission', 'vent_during_hosp', 'vent',
               'dialysis', 'vasopressor', 'mortality', 'actualicumortality']



In [209]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

mytable = TableOne(df, columns=columns, categorical=categorical, groupby='hospitaldischargeyear')
mytable

Grouped by hospitaldischargeyear                                            
                                                                          Missing       Overall           2014           2015
n                                                                                         43120          21587          21533
hospitaldischargeyear, n (%)    2014                                            0  21587 (50.1)  21587 (100.0)               
                                2015                                               21533 (49.9)                 21533 (100.0)
age_num, mean (SD)                                                              0   65.2 (15.8)    65.1 (16.0)    65.2 (15.7)
gender, n (%)                                                                   0       1 (0.0)        1 (0.0)               
                                Female                                             19929 (46.2)    9955 (46.1)    9974 (46.3)
                                Male                                               23185 (53.8)   11628 (53.9)   11557 (53.7)
                                Unknown                                                 5 (0.0)        3 (0.0)        2 (0.0)
teachingstatus, n (%)           False                                           0  30173 (70.0)   14820 (68.7)   15353 (71.3)
                                True                                               12947 (30.0)    6767 (31.3)    6180 (28.7)
numbedscategory, n (%)          100 - 249                                    4215   8062 (20.7)    3983 (20.4)    4079 (21.0)
                                250 - 499                                          10653 (27.4)    5274 (27.0)    5379 (27.7)
                                <100                                                 2220 (5.7)     1198 (6.1)     1022 (5.3)
                                >= 500                                             17970 (46.2)    9047 (46.4)    8923 (46.0)
ethnicity, n (%)                                                                0     284 (0.7)      124 (0.6)      160 (0.7)
                                African American                                    4866 (11.3)    2399 (11.1)    2467 (11.5)
                                Asian                                                 748 (1.7)      411 (1.9)      337 (1.6)
                                Caucasian                                          32707 (75.9)   16286 (75.4)   16421 (76.3)
                                Hispanic                                             2313 (5.4)     1286 (6.0)     1027 (4.8)
                                Native American                                       270 (0.6)      136 (0.6)      134 (0.6)
                                Other/Unknown                                        1932 (4.5)      945 (4.4)      987 (4.6)
has_sepsis, n (%)               0                                               0  34892 (80.9)   17412 (80.7)   17480 (81.2)
                                1                                                   8228 (19.1)    4175 (19.3)    4053 (18.8)
cancer_hemato, n (%)            0                                               0  31664 (73.4)   15659 (72.5)   16005 (74.3)
                                1                                                  11456 (26.6)    5928 (27.5)    5528 (25.7)
cancer_non_hemato, n (%)        0                                               0  10236 (23.7)    5291 (24.5)    4945 (23.0)
                                1                                                  32884 (76.3)   16296 (75.5)   16588 (77.0)
metastasis, n (%)               0                                               0  39472 (91.5)   19842 (91.9)   19630 (91.2)
                                1                                                    3648 (8.5)     1745 (8.1)     1903 (8.8)
renal_GU, n (%)                 0                                               0  29777 (69.1)   14627 (67.8)   15150 (70.4)
                 

In [210]:
print(mytable.tabulate(tablefmt = "fancy_grid"))


╒═════════════════════════════════╤══════════════════╤═══════════╤══════════════╤═══════════════╤═══════════════╕
│                                 │                  │ Missing   │ Overall      │ 2014          │ 2015          │
╞═════════════════════════════════╪══════════════════╪═══════════╪══════════════╪═══════════════╪═══════════════╡
│ n                               │                  │           │ 43120        │ 21587         │ 21533         │
├─────────────────────────────────┼──────────────────┼───────────┼──────────────┼───────────────┼───────────────┤
│ hospitaldischargeyear, n (%)    │ 2014             │ 0         │ 21587 (50.1) │ 21587 (100.0) │               │
├─────────────────────────────────┼──────────────────┼───────────┼──────────────┼───────────────┼───────────────┤
│                                 │ 2015             │           │ 21533 (49.9) │               │ 21533 (100.0) │
├─────────────────────────────────┼──────────────────┼───────────┼──────────────┼───────

In [211]:
mytable.to_excel('dataDrift_table1_eicu_June20.xlsx')
